In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

## Generating Shakesperian Text using a Character RNN

We'll start exploring NLP by creating an RNN that takes in a sequence of characters and tries to predict the next character in the sequence. Check this [cool article](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) that contains examples (including Algebraic Geometry)!

We'll train our model on Shakespeare's text

#### Creating the training set

In [3]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [4]:
shakespeare_text[:15]

'First Citizen:\n'

To process the text, we'll encode each character as an integer by using Keras' ```Tokenizer``` class

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

Now we can give the tokenizer a string and get back its encoding

In [6]:
tokenizer.texts_to_sequences([shakespeare_text[:15]])

[[20, 6, 9, 8, 3, 1, 19, 6, 3, 6, 36, 2, 10, 24, 11]]

In [7]:
max_id = len(tokenizer.word_index)  # number of distinct characters
max_id

39

In [8]:
# total number of characters
dataset_size = tokenizer.document_count
dataset_size

1115394

Encode the text (subtracting 1 to get ranges from 0 to 38)

In [9]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [10]:
encoded

array([19,  5,  8, ..., 20, 26, 10])

#### Splitting a sequential Dataset

Now we need to split into training, validation and test set. But how we do it with characters? We can't just shuffle all the characters in the text.

With time series, we gnerally split on time: for example if we might take years 2000 to 2012 for training, 2013 to 2015 for validation and 2016 to 2018 for test.

In some other case we can split among other dimensions such as an industry type or country.

Splitting accross time is safe, however it implicitly assumes that patterns learned in the past will still exist in the future. I.e. we assume the time series is *stationary*. To make sure the time series is sufficiently stationary, we can plot the model's error on the validation set across time: if the model performes better on the first part of the validation set than on the last part, then the time series might not be stationary enough. Training the model on a shorter time span might be better.

For the Shakespeare example  we'll take the first 90% of the text for the training set keeping the rest for validation and test

In [11]:
train_size = dataset_size *90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

#### Chopping data into windows

We'll use ```dataset.window``` to convert the one long instance we have into many smaller windows of text. Every instance in training will be a fairly short substring of the full text and the RNN will be unrolled over the length of the substrings. This is called **truncated backpropagation through time**

In [12]:
n_steps = 100 # tune-able parameter
window_length = n_steps + 1 # target will be the input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

Using ```shift=1``` we create overlapping windows: the first window will contain characters 0 to 100, the second 1 to 101 and so on...

```drop_remainder``` ensures all windows are 101 characters long and we don't have to do any padding

```window()``` creates a dataset containing windows, each of which is also a dataset. It is a *nested dataset*. This is useful when we want to transform each window by calling dataset methods but it cannot be used directly for training. The model expects tensors, not datasets. 

The ```flat_map``` dataset converts a nested dataset into a *flat dataset* (i.e. without nesting). It also takes a function as an argument which allows you to transform each dataset _before_ flattening.

For example passing ```lambda ds: ds.batch(2)``` to ```flat_map``` the dataset $\{\{1, 2\}, \{3,4, 5,6\}\}$ becomes $\{[1,2], [3,4], [5,6]\}$.

In [13]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

By calling passing ```window_length``` to batch, we get a single tensor for each batch (all windows have that length). That is, the dataset contains windows of 101 characters each. Now we can shuffle the tensors and separate inputs from targets. See Figure 16-1 for a pictorial representation of the process

In [14]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [15]:
for X, y in dataset.take(1):
    print(X)
    print(y)

tf.Tensor(
[[ 4  2  0 ...  2  0  9]
 [10  2  6 ... 23 10  1]
 [ 9 20  0 ...  6  4 25]
 ...
 [ 6  3 13 ... 15 17  0]
 [ 0  3 19 ...  0  3  9]
 [ 9  5 13 ... 17  0  4]], shape=(32, 100), dtype=int64)
tf.Tensor(
[[ 2  0  6 ...  0  9  3]
 [ 2  6  1 ... 10  1  5]
 [20  0  2 ...  4 25  1]
 ...
 [ 3 13 20 ... 17  0  7]
 [ 3 19  0 ...  3  9  1]
 [ 5 13  7 ...  0  4  9]], shape=(32, 100), dtype=int64)


We'll do one-hot-encoding since there are fairly few features (only 39 characters)

In [16]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

#### Bulding and Training the Char-RNN model

We'll create a model with GRU units that are fed into a Time-distributed Dense layer of 39 unites (max_id), since we want to output character probabilities for each possible character

**Note:** Running the code below takes a good while...

In [17]:
# model = keras.models.Sequential([
#     keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
#                      dropout=0.2, recurrent_dropout=0.2),
#     keras.layers.GRU(128, return_sequences=True,
#                      dropout=0.2, recurrent_dropout=0.2),
#     keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
# ])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
# history = model.fit(dataset, epochs=3)
# model.save('saved_models/16_nlp_with_rnns/shakespeare.h5')

In [19]:
model = keras.models.load_model('saved_models/16_nlp_with_rnns/shakespeare.h5')

#### Using the Char-RNN model

To pass some inputs to our model, we need to apply the same preprocessing we did for training it

In [20]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

And a test:

In [21]:
X_new = preprocess(['How are yo'])
y_pred = np.argmax(model.predict(X_new), axis=-1) + 1 # Account for the 1 we removed during preprocessing
y_pred

array([[ 2, 12,  1,  6, 10,  2,  1, 16,  4, 14]])

The model generate one predicion for each character in the input. Since we're only interested in the prediction for the last character we simply pick up the last prediction

In [22]:
tokenizer.sequences_to_texts(y_pred)[0][-1] # 1st sentence, last char

'u'

#### Generating new text

Now that we have a model, we could start using it by feeding in some text, generate a prediction, amend the text, feed it in, get a new prediction, etc...

While this approach does work, it often leads to the same words being predicted over and over again. Instead, we can sample from the distribution of possible characters using ```tf.random.categorical()```, this will generate more interesting text.

```categorical``` samples random class indices, given class log probabilities. We can also introduce a notion of *temperature*, which will control how wild our model will deviate from the logits. We divide the logits by the temperature, when temperature is close to 0 it will favour high probability characters, when it is close to 1, it gives characters an equal probability.

We'll define the ```next_char``` function to do this for us

In [23]:
def next_char(text, model, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

And a helper function that will generate the text and fill it in

In [24]:
def complete_text(text, model, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text+= next_char(text, model, temperature)
    return text

In [25]:
print(complete_text('t', model, temperature=0.5))

troth the head of a scolding winds.

grumio:
i will


In [26]:
print(complete_text('test', model, temperature=0.5))

test best,
that he was a schoolmaster, sir, then that 


In [27]:
print(complete_text('adam', model, temperature=0.8))

adam have i pawry of rag.

tranio:
why, though so can 


In [28]:
print(complete_text('u', model, temperature=1))

ur beautiful will i do take,
and let me baptista fa


In [29]:
print(complete_text('k', model, temperature=2))

ks!
far to pray youught go;
be, take you: still you


To get better text we can add more layers, train for longer, add regularization. The limitation of this model is that it is incapable of learning patterns longer than n_steps=100 characters. Stateful RNNs can help with that.

#### Stateful RNN

The RNNs discussed so far only used hidden states at each epoch. If we preserve the state after the end of each step and use that as the starting point for the next step, the RNN willl learn long-term patterns despite only backpropagating through short sequences. This is called a *stateful RNN*.

It has a catch though: training a stateful RNN only makes sense if each input sequence in a batch starts exactly where the corresponding sequence in the previous batch left off.

Instead of using shuffled data, we will use sequential, non-overlapping sequences. To do this we'll create the dataset using ```shift=n_steps``` and naturally we won't ```shuffle``` the data. 

Batching is where things get a bit harder. If we were to use ```batch(32)``` then the first batch would contain windows 1 through 32 and the next batch would be 33 to 64, so the first window of each batch are not consecutive. One approach to this is to use 'batches' containing a single window

In [30]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

Another approach would be to divide the corpus into 32 texts of equal length, create one dataset of consecutive input sequences for them and then use ```tf.data.Dataset.zip(dataset).map(lambda *windows: tf.stack(windows))``` to create proper batches, where the nth input sequence in a batch start off exactly where the nth input sequence ended in the previous batch

In [31]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

Now to create the RNN we must set its ```stateful``` parameter as wel as specify the batch_input_shape for the first layer. We leave the second dimension unspecified since the inputs can have any length

In [32]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

We also need to add a little callback to ensure we reset the states at the end of each epoch.

In [33]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [81]:
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
# history = model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])
# model.save('saved_models/16_nlp_with_rnns/stateful_shake.h5')

Epoch 1/50
313/313 [==============================] - 63s 200ms/step - loss: 1.6431
Epoch 2/50
313/313 [==============================] - 62s 199ms/step - loss: 1.6372
Epoch 3/50
313/313 [==============================] - 63s 201ms/step - loss: 1.6314
Epoch 4/50
313/313 [==============================] - 63s 202ms/step - loss: 1.6261
Epoch 5/50
313/313 [==============================] - 63s 202ms/step - loss: 1.6208
Epoch 6/50
313/313 [==============================] - 63s 202ms/step - loss: 1.6164
Epoch 7/50
313/313 [==============================] - 63s 202ms/step - loss: 1.6123
Epoch 8/50
313/313 [==============================] - 63s 202ms/step - loss: 1.6097
Epoch 9/50
313/313 [==============================] - 66s 211ms/step - loss: 1.6059
Epoch 10/50
313/313 [==============================] - 65s 209ms/step - loss: 1.6021
Epoch 11/50
313/313 [==============================] - 64s 203ms/step - loss: 1.5980
Epoch 12/50
313/313 [==============================] - 64s 203ms/step - lo

In [34]:
model = keras.models.load_model('saved_models/16_nlp_with_rnns/stateful_shake.h5')

Now that this model is trained, it can only make predictions of the same size as the batch size used during training. To avoid this restriction, we can create a statelss model and copy the stateful model weights' to this model

In [35]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

In [36]:
stateless_model.set_weights(model.get_weights())

And then use it for some predictions!

In [39]:
print(complete_text('u', stateless_model))

ud there:
haste a son thou affect not paul to that:


In [40]:
print(complete_text('adam', stateless_model, temperature=0.8))

adament
to expect and time and unto the man from thour


## Sentiment Analysis

For this we'll use the IMDB reviews dataset

In [44]:
help(keras.datasets.imdb.load_data)

Help on function load_data in module tensorflow.python.keras.datasets.imdb:

load_data(path='imdb.npz', num_words=None, skip_top=0, maxlen=None, seed=113, start_char=1, oov_char=2, index_from=3, **kwargs)
    Loads the [IMDB dataset](https://ai.stanford.edu/~amaas/data/sentiment/).
    
    This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment
    (positive/negative). Reviews have been preprocessed, and each review is
    encoded as a list of word indexes (integers).
    For convenience, words are indexed by overall frequency in the dataset,
    so that for instance the integer "3" encodes the 3rd most frequent word in
    the data. This allows for quick filtering operations such as:
    "only consider the top 10,000 most
    common words, but eliminate the top 20 most common words".
    
    As a convention, "0" does not stand for a specific word, but instead is used
    to encode any unknown word.
    
    Arguments:
        path: where to cache the data (relativ

In [42]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

17465344/17464789 [==============================] - 6s 0us/step


In [45]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

From the description, the dataset has already been transformed, with each character being encoded by its character frequency rank.

The integers 0, 1, 2 are special and represent padding token, Start of Sequense (SoS) token and unknown words, respectively. Let's write a function to decode the text

In [47]:
word_index = keras.datasets.imdb.get_word_index()

1646592/1641221 [==============================] - 1s 0us/step


In [53]:
def decode(text):
    id_to_word = {id_+3: word for word, id_ in word_index.items()}
    for id_, token in enumerate(('<pad>', '<SoS>', '<unk>')):
        id_to_word[id_] = token
    return " ".join(id_to_word[c] for c in text)

In [55]:
decode(X_train[0][:100])

"<SoS> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was"

In a real project we'd have to take care of preprocessing ourselves. Kera's Tokenizer works well for English and most languages but there are exceptions (see pg.535), for example words like 'San Francisco' or '#ILoveDeepLearning'. 

One solution came in a paper by [Taku Kudu](https://arxiv.org/abs/1804.10959) that used unsupervised learning techniques to tokenize and detokenize text at subword level in a language independent way. Google's [SentencePie](https://github.com/google/sentencepiece) is an implementation of this.

[TF.Text](https://medium.com/tensorflow/introducing-tf-text-438c8552bd5e) contains various tokenization strategies, including [WordPiece](https://arxiv.org/abs/1609.08144).

We'll explore some TF operations to do preprocessing by loading the original IMDB reviews dataset.

In [67]:
import tensorflow_datasets as tfds

datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples

In [60]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)  # Truncate, keep only first 300 chars -> speed up training
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")  # Replaces <br /> with spaces
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")  # remove anything other than characters or ''
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch
                                       

The next step is to construct the vocabulary by going through the dataset and keeping track of word occurences

In [61]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets['train'].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [62]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

We'll truncate the vocabulary to the 10,000 most common words

In [64]:
vocab_size = 10_000
truncated_vocabulary = [word for word, count in  vocabulary.most_common()[:vocab_size]]

Then replace each word with it's ID from the vocabulary

In [65]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

This table will serve as our reference for a given input. In the example below, faaaantastic is an unseen word so it falls into a newly created bucket

In [66]:
table.lookup(tf.constant([b"This move was faaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,   951,    11, 10791]])>

**Tip:** check out TF-Transform, in particular ```tft.compute_and_apply_vocabulary```, it goes through the dataset to find vocabulary and generate TF operations required to encode each word with the vocab.

Now to create the final training set we use the previously defined ```preprocess``` and encode them with the vocabulary we just built

In [68]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets['train'].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [70]:
for X, y in train_set.take(1):
    print(X)
    print(y)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   70 ...    0    0    0]
 [4099 6881    1 ...    0    0    0]
 ...
 [  22   12  118 ...  331 1047    0]
 [1757 4101  451 ...    0    0    0]
 [3365 4392    6 ...    0    0    0]], shape=(32, 60), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


And train!

In [69]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 100s 128ms/step - loss: 0.6684 - accuracy: 0.5717
Epoch 2/5
782/782 [==============================] - 100s 128ms/step - loss: 0.4626 - accuracy: 0.7879
Epoch 3/5
782/782 [==============================] - 102s 131ms/step - loss: 0.2951 - accuracy: 0.8833
Epoch 4/5
782/782 [==============================] - 100s 128ms/step - loss: 0.1737 - accuracy: 0.9407
Epoch 5/5
782/782 [==============================] - 105s 135ms/step - loss: 0.1273 - accuracy: 0.9558


The first layer we use is an Embedding layer that will convert word IDs into embeddings. It's input matrix has one row per id (vocab_size + num_oov_buckets) and a column per embedding dimension (used 128 but this is a hyperparameter). The embedding layer outputs a 3D tensor of shape **[batch_size, time_steps, embedding_size]** which feeds in perfectly into the rest of the network.

### Masking

This model will have to learn to ignore padding tokens (which have index zero). We can set the Embedding layer's ```mask_zero``` parameter to True to make it pad tokens whose ID is 0. (note: in this case, the pad token has index zero because it is the most common token, in general is a good idea to map padding to 0). With this, padding is ignored by the layers downstream!

With this parameter set to true, the Embedding layer creates a *mask tensor* equal to K.not_equal(inputs, 0); a boolean tensor with entries equal to False where inputs==0, True otherwise. This mask is propagated downstream as long as the time dimension is preserved. For example, the first GRU layer outputs sequences so it preserves the mask. The second doesn't so the mask is not transmitted to the dense layer. Each layer behaves differently when encountering a masked time step, but in general it simple ignored masked time steps.

For example, recurrent layers simply copies the output from previous time step when it encounters a mask. If the mask propagates all the way to the output (e.g. a vec-to-seq or seq-to-seq architecture), then the masked time steps will not contribute to the loss.

Note: LSTMs and GRUs are optimized for GPUs based on Nvidia's cuDNN library, however this implementation does not support masking. The optimized implementation requires you to use the default values for several hyperparameters: activation, recurrent_activation, etc..

All layers that receive the mask must support masking (else an exception is raised). All recurrent layers, as well as TimeDistributed and a few other layers support this. 

See pg 539 for more info

Masking works best for simple sequential models, it doesn't always work for more complex models (e.g. if you need to mix a Conv1D layer with recurrent layers). In such cases you will need to explicitly compute the mask. 

Below an example that is the same model defined as above, but done using the Functional API and handling masks manually

In [76]:
import os
root_logdir = os.path.join(os.curdir, 'my_logs')

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

callbacks = [keras.callbacks.TensorBoard(get_run_logdir())]

In [77]:
K = keras.backend
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation='sigmoid')(z)

model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5, callbacks=callbacks)

Epoch 1/5
782/782 [==============================] - 125s 159ms/step - loss: 0.5336 - accuracy: 0.7226
Epoch 2/5
782/782 [==============================] - 113s 144ms/step - loss: 0.3419 - accuracy: 0.8588
Epoch 3/5
782/782 [==============================] - 110s 141ms/step - loss: 0.1786 - accuracy: 0.9352
Epoch 4/5
782/782 [==============================] - 108s 138ms/step - loss: 0.1334 - accuracy: 0.9516
Epoch 5/5
782/782 [==============================] - 112s 143ms/step - loss: 0.1107 - accuracy: 0.9592


In [79]:
test_set = datasets['test'].batch(32).map(preprocess)
test_set = test_set.map(encode_words).prefetch(1)

Let's generate predictions for 1 batch

In [90]:
for batch_X, batch_y in test_set.take(1):
    y_preds = np.round(model.predict(test_set.take(1))).flatten()
    print('True values\n', batch_y)
    print('\nPredictions\n', y_preds)

True values
 tf.Tensor([1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 1 1 1], shape=(32,), dtype=int64)

Predictions
 [1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 1. 1. 0.]


Not too shabby, now let's look at Pretrained Embeddings!

### Reusing Pretrained Embeddings

Using pre-trained models is pretty easy with tensorflow_hub. For this example we'll use ```nnlm-en-dim50``` sentence embedding module version 1. 

This module is a sequence encoder, taking strings as inputs and encoding each as a single vector (a 50dim vector). It parses the strings and embeds each word using an embedding matrix trained on the Google News 7B corpus, it then compute the mean of all the word embeddings and the result is the sentence embedding.

By default ```hub.KerasLayer``` is not trainable but we can set ```training=True``` to make it trainable.

In [100]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

Reload the dataset before training

In [101]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].repeat().batch(batch_size).prefetch(1)
history = model.fit(train_set, steps_per_epoch=train_size // batch_size, epochs=5,
                    callbacks=[keras.callbacks.TensorBoard(get_run_logdir())])

Epoch 1/5
781/781 [==============================] - 228s 291ms/step - loss: 0.5480 - accuracy: 0.7246
Epoch 2/5
781/781 [==============================] - 221s 284ms/step - loss: 0.5134 - accuracy: 0.7473
Epoch 3/5
781/781 [==============================] - 223s 285ms/step - loss: 0.5083 - accuracy: 0.7512
Epoch 4/5
781/781 [==============================] - 223s 285ms/step - loss: 0.5049 - accuracy: 0.7538
Epoch 5/5
781/781 [==============================] - 223s 285ms/step - loss: 0.5022 - accuracy: 0.7555


## An Encoder-Decoder Network for Neural Machine Translation